In [1]:
import pandas as pd

import re

from sqlalchemy import create_engine

In [2]:
# Ouverture de la connexion vers la BDD
engine = create_engine('sqlite:///data.sqlite')
connection = engine.connect()

In [3]:
df = pd.read_excel('base-ic-evol-struct-pop-2014.xls', header=5,
                   dtype={'REG': str})

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50094 entries, 0 to 50093
Data columns (total 84 columns):
IRIS              50094 non-null object
REG               50094 non-null object
DEP               50094 non-null object
UU2010            50094 non-null object
COM               50094 non-null object
LIBCOM            50094 non-null object
TRIRIS            50094 non-null object
GRD_QUART         50094 non-null object
LIBIRIS           50094 non-null object
TYP_IRIS          50094 non-null object
MODIF_IRIS        50094 non-null int64
LAB_IRIS          50094 non-null object
P14_POP           50094 non-null float64
P14_POP0002       50094 non-null float64
P14_POP0305       50094 non-null float64
P14_POP0610       50094 non-null float64
P14_POP1117       50094 non-null float64
P14_POP1824       50094 non-null float64
P14_POP2539       50094 non-null float64
P14_POP4054       50094 non-null float64
P14_POP5564       50094 non-null float64
P14_POP6579       50094 non-null float64
P1

In [4]:
df.sample(5).iloc[:, 9:]

,TYP_IRIS,MODIF_IRIS,LAB_IRIS,P14_POP,P14_POP0002,P14_POP0305,P14_POP0610,P14_POP1117,P14_POP1824,P14_POP2539,...,C14_F15P_CS4,C14_F15P_CS5,C14_F15P_CS6,C14_F15P_CS7,C14_F15P_CS8,P14_POP_FR,P14_POP_ETR,P14_POP_IMM,P14_PMEN,P14_PHORMEN
7200,Z,0,5,340.000000,7.976540,8.973607,12.961877,34.897361,20.938416,33.900293,...,3.988270,35.894428,3.988270,67.800587,43.870968,297.126100,42.873900,40.879765,340.000000,0.0
21980,Z,0,5,170.000000,3.756906,5.635359,7.513812,8.453039,6.574586,25.359116,...,9.392265,14.088398,4.696133,23.480663,18.784530,161.546961,8.453039,15.027624,170.000000,0.0
2031,Z,0,5,461.000000,11.000000,22.000000,23.000000,34.000000,18.000000,66.000000,...,25.000000,35.000000,0.000000,45.000000,45.000000,458.000000,3.000000,12.000000,453.000000,8.0
48002,H,0,1,1761.795351,61.789156,64.723008,81.750292,140.788277,139.846214,500.629670,...,165.371854,149.769511,33.454457,130.166928,165.640356,1459.995608,301.799744,362.381160,1761.795351,0.0
23316,Z,0,5,432.000000,22.102326,23.106977,33.153488,39.181395,29.134884,81.376744,...,50.232558,20.093023,5.023256,55.255814,25.116279,430.995349,1.004651,3.013953,432.000000,0.0


In [5]:
ll = []

for c in df.columns:
    if re.match('P14_POP[0-9]{4}', c) or re.match('P14_POP[0-9]{2}P', c):
        ll.append(c)

print(ll)

['P14_POP0002', 'P14_POP0305', 'P14_POP0610', 'P14_POP1117', 'P14_POP1824', 'P14_POP2539', 'P14_POP4054', 'P14_POP5564', 'P14_POP6579', 'P14_POP80P', 'P14_POP0014', 'P14_POP1529', 'P14_POP3044', 'P14_POP4559', 'P14_POP6074', 'P14_POP75P', 'P14_POP0019', 'P14_POP2064', 'P14_POP65P']


In [13]:
# Transposition
dft = pd.melt(
    df,
    id_vars=['IRIS'],
    value_vars=ll,
    var_name='Age',
    value_name='Population'
)

# Nettoyage de la colonne Age
dft['Age'] = dft['Age'].str.replace('P14_POP', '')

# Export dans SQL
dft.to_sql('pop_ages', connection, if_exists='replace', index=False)

In [5]:
# Création de la table IRIS
df[['IRIS', 'COM', 'LIBIRIS']].to_sql('iris', connection,
                                      if_exists='replace', index=False)

In [6]:
# Création de la table communes
df[['COM', 'LIBCOM']].drop_duplicates().to_sql('communes', connection,
                                               if_exists='replace',
                                               index=False)

In [7]:
connection.close()

,IRIS,REG,DEP,UU2010,COM,LIBCOM,TRIRIS,GRD_QUART,LIBIRIS,TYP_IRIS,...,C14_F15P_CS4,C14_F15P_CS5,C14_F15P_CS6,C14_F15P_CS7,C14_F15P_CS8,P14_POP_FR,P14_POP_ETR,P14_POP_IMM,P14_PMEN,P14_PHORMEN
0,010010000,84,01,01000,01001,L'Abergement-Clémenciat,ZZZZZZ,0100100,L'Abergement-Clémenciat (commune non irisée),Z,...,50.000000,85.000000,30.000000,70.000000,15.000000,759.000000,8.000000,20.000000,767.000000,0.000000
1,010020000,84,01,01000,01002,L'Abergement-de-Varey,ZZZZZZ,0100200,L'Abergement-de-Varey (commune non irisée),Z,...,19.670782,24.588477,0.000000,19.670782,0.000000,237.032922,1.967078,2.950617,239.000000,0.000000
2,010040101,84,01,01302,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Les Perouses-Triangle d'Activite,H,...,72.108846,142.061017,29.075716,214.857349,182.069302,1481.872018,245.605128,274.052131,1391.485087,335.992058
3,010040102,84,01,01302,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Longeray-Gare,H,...,183.181992,402.998978,168.120565,401.086316,345.384322,3313.368248,418.945924,531.318960,3732.314172,0.000000
4,010040201,84,01,01302,01004,Ambérieu-en-Bugey,ZZZZZZ,0100402,Centre-St Germain-Vareilles,H,...,213.949238,434.011620,126.275371,535.665841,309.729363,3639.595835,276.314331,325.320938,3827.912246,87.997920
5,010040202,84,01,01302,01004,Ambérieu-en-Bugey,ZZZZZZ,0100402,Tiret-Les Allymes,H,...,379.106396,401.552589,107.054790,480.798848,334.682300,4552.806487,93.492030,236.275511,4595.299722,50.998795
6,010050000,84,01,01000,01005,Ambérieux-en-Dombes,ZZZZZZ,0100500,Ambérieux-en-Dombes (commune non irisée),Z,...,120.592959,216.062384,40.197653,150.741198,85.420012,1604.891291,22.108709,63.311303,1627.000000,0.000000
7,010060000,84,01,01000,01006,Ambléon,ZZZZZZ,0100600,Ambléon (commune non irisée),Z,...,9.909091,9.909091,4.954545,14.863636,0.000000,102.063636,6.936364,7.927273,109.000000,0.000000
8,010070000,84,01,01000,01007,Ambronay,ZZZZZZ,0100700,Ambronay (commune non irisée),Z,...,184.000000,285.000000,75.000000,235.000000,175.000000,2528.000000,42.000000,88.000000,2565.000000,5.000000
9,010080000,84,01,01302,01008,Ambutrix,ZZZZZZ,0100800,Ambutrix (commune non irisée),Z,...,59.919355,95.870968,23.967742,55.924731,31.956989,725.024194,17.975806,22.969086,743.000000,0.000000
